In [140]:
from marshmallow import fields, Schema as MarshMallowSchema, post_load

In [142]:
class Schema(MarshMallowSchema):
    pass


class Response(Schema):
    foo = fields.String()

    
Response().load({"foo": "bar"})

{'foo': 'bar'}